딥 러닝을 이용한 자연어 처리 입문(https://wikidocs.net/book/2155)의 위키독스를 참고하여 정리중입니다. 


## 자연어처리(Natural Language Processing, NLP)


- 자연어란 우리가 평소에 말하는 음성이나 텍스트를 말한다. 즉 **자연어 처리**는 이러한 음성이나 텍스트를 컴퓨터가 인식하고 처리하는 것을 말한다.

자연어 처리는 음성 인식, 내용 요약, 번역, 사용자의 감성 분석, 텍스트 분류 작업(스팸 메일 분류, 뉴스 기사 카테고리 분류), 질의 응답 시스템, 챗봇과 같은 곳에서 활용되고 있는데, 딥러닝이 등장하면서 대용량 데이터를 학습할 수 있게 되었고 그에따라 자연어 처리 연구가 활발해지기 시작했다.



**컴퓨터 알고리즘은 수치로 된 데이터만 이해할 뿐 텍스트를 이해할 수 없기 때문에 텍스트 자료를 딥러닝에 그대로 입력할 수 있는건 아니다.**

그에따라 텍스트를 정제하는 전처리 과정이 필요하며 이제 해당 과정을 정리해볼까 한다.(우선 한글 기준으로)

## 1. 텍스트의 토큰화(word tokenization)

자연어처리에서는 텍스트를 단어별, 문장별, 형태소별로 나눠 구분하는데 이렇게 나누어진 하나의 단위를 **토큰(token)**이라고 한다.

이렇게 토큰별로 나누는 과정을 **토큰화(tokenization)**라고 부른다.

직접 다음 문장을 나눠보자.

***
**'나는 자연어 처리를 능숙하게 숙달할 것이다...!!!'**
***

케라스에서 제공하는 text 모듈의 text_to_word_sequence() 함수를 사용하면 문장을 단어 단위로 쉽게 나눌 수 있다.

In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text = '나는 자연어 처리를 능숙하게 숙달할 것이다...!!!'
result = text_to_word_sequence(text)
print(result)

['나는', '자연어', '처리를', '능숙하게', '숙달할', '것이다']


토큰의 기준을 단어(word)로 하는 경우 **단어 토큰화**라고 하며, 이때 단어(word)는 **단어 단위 외에도 단어구, 의미를 갖는 문자열**로도 간주된다. 따라서 앞선 문장의 단어 토큰화 작업에서는 띄어쓰기를 기준으로 마침표(.)나 느낌표(.)같은 구두점이 제외된 것을 볼 수 있다.


그러나, 의미가 없다고 해도 구두점을 반드시 제외해야한다는 생각은 옳지 않은데 다음 문장을 예를 들어보자.
***
**Walk on, walk on
With hope in your heart
And you'll never walk alone,
You'll never walk alone.**
***

영어 문장에는 보통 아포스트로피(')가 들어가는 경우가 대다수다. **NLTK**는 영어 코퍼스를 토큰화하기 위한 도구들을 제공하는데, 
word_tokenize와 WordPunctTokenizer를 사용하여 어떻게 해석하는지 살펴보자.

In [2]:
from nltk import word_tokenize

print(word_tokenize("Walk on, walk on With hope in your heart And you'll never walk alone You'll never walk alone."))

['Walk', 'on', ',', 'walk', 'on', 'With', 'hope', 'in', 'your', 'heart', 'And', 'you', "'ll", 'never', 'walk', 'alone', 'You', "'ll", 'never', 'walk', 'alone', '.']


In [3]:
from nltk import WordPunctTokenizer
print(WordPunctTokenizer().tokenize("Walk on, walk on With hope in your heart And you'll never walk alone You'll never walk alone"))

['Walk', 'on', ',', 'walk', 'on', 'With', 'hope', 'in', 'your', 'heart', 'And', 'you', "'", 'll', 'never', 'walk', 'alone', 'You', "'", 'll', 'never', 'walk', 'alone']


공통적으로 nltk는 케라스와 달리 구두점을 보존하며, **word_tokenize**는 you'll을 따로 토큰화했으며 **WordPunctTokenizer**는 '까지 토큰화했다.

In [4]:
text = "Walk on, walk on With hope in your heart And you'll never walk alone You'll never walk alone"
result = text_to_word_sequence(text)
print(result)

['walk', 'on', 'walk', 'on', 'with', 'hope', 'in', 'your', 'heart', 'and', "you'll", 'never', 'walk', 'alone', "you'll", 'never', 'walk', 'alone']


케라스의 **text_to_word_sequence**는 구두점도 없애고 대문자를 소문자로 변환하여 출력했다.

## 토큰화시 고려사항


### 1. 구두점이나 특수 문자를 무조건 제거하지 말 것

- 마침표(.)를 문장의 경계를 구분할 때 사용할 수 있고, Ph.D나 ($)15000 등 특수 문자가 의미를 가질수도 있기 때문이다.

### 2. 줄임말과 단어 내에 띄어쓰기가 있는 경우

- 영어권에서는 what's가 what in인 것과 같이 아포스트로피(')를 사용하여 압축된 단어를 다시 펼치는 역할을 할 때가 있다.<br>

- 띄어쓰기의 경우에도 rock 'n' roll의 경우 하나의 단어 사이에 띄어쓰기가 있는 경우에도 하나의 토큰으로 봐야하는 경우가 있기 때문에, 토큰화 작업은 저런 단어를 하ㄴ로 인식하도록 해야한다.

## 문장 토큰화(Sentence Tokenization)

- 갖고있는 코퍼스(말뭉치) 내에서 문장 단위로 구분하는 작업으로, **문장 분류(sentence sgmentation)라고도 부른다.**

마침표나 느낌표같은 구두점을 활용해서 문장을 구분할수도 있지만, 구두점으로 항상 문장이 구분되는 것이 아니기에 따로 살펴봐야할 것이다.

영어 문장의 토큰화는 **nltk의 sent_tokenize**를 사용하여 수행할 수 있다. 바로 수행해보자.

#### 영어 문장 토큰화

In [5]:
from nltk.tokenize import sent_tokenize
text = "I need a time. i need a vacation~ i wanna go away! i called S.O.S"
print(sent_tokenize(text))

['I need a time.', 'i need a vacation~ i wanna go away!', 'i called S.O.S']


출력 결과 모든 문장을 성공적으로 구분해 냈다. 마침표를 구분자로 하여 문장을 구분하지 않았기 때문에 Ph.D도 단어로 인식한 것을 볼 수 있다.

#### 한국어 문장 토큰화

한국어에 대한 문장 토큰화는 **KSS(Korean Sentence Splitter)**를 사용하여 수행할 수 있다.

In [6]:
import kss

text = "제가 딥러닝을 공부할 줄은 꿈에도 몰랐습니다. 그래도 재미있어서 계속 진행하려고 합니다! 그런데 지금이야 재밌지 나중에도 재미있을까요? 일단 많이 어렵더라도 열심히 해보겠습니다!"

print(kss.split_sentences(text))

['제가 딥러닝을 공부할 줄은 꿈에도 몰랐습니다.', '그래도 재미있어서 계속 진행하려고 합니다!', '그런데 지금이야 재밌지 나중에도 재미있을까요?', '일단 많이 어렵더라도 열심히 해보겠습니다!']


## 한국어에서의 토큰화의 어려움.

영어는 거의 대부분의 경우에서 단어 단위로 띄어쓰기가 이뤄지기 때문에 New York와 같은 합성어나 he's와 같은 줄임말에 대한 예외처리만 한다면, 띄어쓰기를 기준으로 하는 띄어쓰기 토큰화를 수행해도 단어 토큰화가 잘 작동한다.

그러나 한국어는 영어와는 달리 한국어는 띄어쓰기 단위가 되는 단위 즉, **어절** 토큰화는 지양한다.
그 근본적인 이유는 한국어가 영어와는 다른 형태를 가지는 **교착어**라는 점이기 때문이다.

**교착어란 조사, 어미 등을 붙여서 말을 만드는 언어를 일컫는다.**

### 1. 한국어는 교착어이다.

한국어는 영어와 달리 조사라는 것이 존재하는데, '그'(he/him)라는 주어나 목적어가 들어간 문장이 있다고 하자.
'그'라는 단어 하나에도 '그가', '그에게', '그를', '그와', '그는'과 같이 다양한 조사가 '그'라는 글자 뒤에 띄어쓰기 없이 붙는다. 따라서, 대부분의 한국어 자연어처에서는 조사를 분리해줄 필요가 있다.


한국어 토큰화에서는 **형태소(morpheme)**란 개념을 반드시 이해해야 하는데, **형태소란 뜻을 가진 가장 작은 말의 단위를 말한다.** 

#### 형태소의 종류
***
**자립 형태소** : 접사, 어미, 조사와 상관없이 자립하여 사용할 수 있는 형태소, 그 자체로 단어가 되며, 체언(명사, 대명사, 수사), 수식언(관형사, 부사), 감탄사 등이 있다.

**의존 형태소** : 다른 형태소와 결합하여 사용되는 형태소, 접사, 어미, 조사, 어간을 말한다.

> **예시 문장 : 나는 딥러닝책을 읽었다.**
<br><br>
예시 문장을 형태소 단위로 분해
<br><br>
자립 형태소 : 나, 딥러닝책<br>
의존 형태소 : -는, -을, 읽-, -었, -다
***
한국어에서는 영어의 단어 토큰화와 유사한 형태를 얻으려면 어절 토큰화가 아닌 형태소 토큰화를 수행해야한다.


### 2. 한국어는 띄어쓰기가 영어보다 잘 지켜지지 않는다.

한국어는 영어보다 띄어쓰기가 어렵고, 띄어쓰기가 지켜지지 않아도 글을 쉽게 이해할 수 있는 언어라는 점에서 잘 지켜지지 않는 경향이 있다. 

***
예제1 : 한국어는붙여도뜻을해석하기용이하다.<br><br>
예제2 : thisisentencewhatismean?
***

## 품사 태깅(Part-of-speech tagging)

단어는 표기는 같지만, 품사에 따라서 단어의 의미가 달라진다. 그에 따라 단어 토큰화 과정에서 **단어의 의미를 제대로 파악하기 위해 각 단어가 어떤 품사로 쓰였는지를 구분**해놓기도 하는데, 이 작업을 **품사 태깅(part-of-speech-tagging)**이라고 한다.

#### 영어 품사 태깅

In [7]:
from nltk.tokenize import word_tokenize
text = 'I need a time. i need a vacation~ i wanna go away! i called S.O.S'
print(word_tokenize(text))

['I', 'need', 'a', 'time', '.', 'i', 'need', 'a', 'vacation~', 'i', 'wan', 'na', 'go', 'away', '!', 'i', 'called', 'S.O.S']


In [11]:
# nltk.tag의 pos_tag를 활용하여 수행
from nltk.tag import pos_tag
x = word_tokenize(text)
pos_tag(x)

[('I', 'PRP'),
 ('need', 'VBP'),
 ('a', 'DT'),
 ('time', 'NN'),
 ('.', '.'),
 ('i', 'VB'),
 ('need', 'VBP'),
 ('a', 'DT'),
 ('vacation~', 'NN'),
 ('i', 'NN'),
 ('wan', 'VBP'),
 ('na', 'TO'),
 ('go', 'VB'),
 ('away', 'RB'),
 ('!', '.'),
 ('i', 'NN'),
 ('called', 'VBD'),
 ('S.O.S', 'NNP')]

**품사 태깅**
***
**PRP** : 인칭 대명사<br>
**VBP** : 동사<br>
**RB** : 부사<br>
**VBG** : 현재부사<br>
**IN** : 전치사<br>
**NNP** : 고유 명사<br>
**NNS** : 복수형 명사<br>
**CC** : 접속사<br>
**DT** : 관사<br>
***

#### 한국어 품사 태깅

한국어 자연어 처리는 **KoNLPy**라는 파이썬 패키지를 사용한다. 해당 패키지를 활용하여  형태소 분석기를 사용할 수 있는데 종류는 다음과 같다.

#### 형태소 분석기 종류
***
1. Okt(Open Korea Text)<br>
2. Mecab<br>
3. Komoran<br>
4. Hannanum<br>
5. Kkma
***

### Okt 사용

In [13]:
from konlpy.tag import Okt
okt = Okt()
text = '안녕하세요 지금은 오후 6시 40분. 한창 배고플 시간입니다.'
print(okt.morphs(text))

['안녕하세요', '지금', '은', '오후', '6시', '40분', '.', '한창', '배고플', '시간', '입니다', '.']


#### okt 품사 태깅

In [14]:
print(okt.pos(text))

[('안녕하세요', 'Adjective'), ('지금', 'Noun'), ('은', 'Josa'), ('오후', 'Noun'), ('6시', 'Number'), ('40분', 'Number'), ('.', 'Punctuation'), ('한창', 'Noun'), ('배고플', 'Adjective'), ('시간', 'Noun'), ('입니다', 'Adjective'), ('.', 'Punctuation')]


#### 명사 추출

In [15]:
print(okt.nouns(text))

['지금', '오후', '한창', '시간']


### Kkma 사용

In [16]:
from konlpy.tag import Kkma
kkma = Kkma()
text = '안녕하세요 지금은 오후 6시 40분. 한창 배고플 시간입니다.'
print(kkma.morphs(text))

['안녕', '하', '세요', '지금', '은', '오후', '6', '시', '40', '분', '.', '하', 'ㄴ', '창', '배고프', 'ㄹ', '시간', '이', 'ㅂ니다', '.']


#### okt 품사 태깅

In [17]:
print(kkma.pos(text))

[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('지금', 'NNG'), ('은', 'JX'), ('오후', 'NNG'), ('6', 'NR'), ('시', 'NNM'), ('40', 'NR'), ('분', 'NNM'), ('.', 'SF'), ('하', 'VV'), ('ㄴ', 'ETD'), ('창', 'NNG'), ('배고프', 'VA'), ('ㄹ', 'ETD'), ('시간', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]


#### 명사 추출

In [18]:
print(kkma.nouns(text))

['안녕', '지금', '오후', '6', '6시', '시', '40', '40분', '분', '창', '시간']
